# Testing Metalearning Predictions script

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [2]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
# library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)
library(zoo)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/Domains_202003/scripts/phaseII_06_fallbacksupp"

In [6]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


# source('../phaseII_03_forest/load_prep_data_expiry.R')



# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)


Attaching package: ‘tidyr’


The following object is masked from ‘package:stringdist’:

    extract




In [41]:
source('functions_metalearning.R')

In [8]:
source('load_prep_data_expiry_2.R')

Loading data...Loaded 5427633 rows
Removing 0 rows due to expiry_date constraints
Removing 758186 rows due to renewed_count constraints
Removing 7716 rows due to missing gibb_score
... now dataset min(creation_date) is  2017-01-01 .


# Fix preds processing into df

In [10]:
fullDir='/home/jupyter/Domains_202003/data/output/models_20201104'
load(file.path(fullDir, 'preds/preds_seg2_rf_ALL.RData'))
load(file.path(fullDir, 'preds/preds_seg2_glm_ALL.RData'))
load(file.path(fullDir, 'preds/preds_seg_rf_ALL.RData'))
load(file.path(fullDir, 'preds/preds_seg_glm_ALL.RData'))
load(file.path(fullDir, 'preds/preds_agg_rf.RData'))
load(file.path(fullDir, 'preds/preds_agg_rf_ALL.RData'))
load(file.path(fullDir, 'preds/preds_agg_glm.RData'))
load(file.path(fullDir, 'preds/preds_agg_glm_ALL.Rdata'))
    

In [11]:
tld_reseller_list1 = expiry_train_df %>%  distinct(tld_registrar_index) %>% pull(tld_registrar_index)
tld_reseller_list2 = expiry_test_df %>%  distinct(tld_registrar_index) %>% pull(tld_registrar_index)
tld_reseller_list = setdiff(tld_reseller_list2,tld_reseller_list1)
tld_reseller_list_ALL = tld_reseller_list

test_list = expiry_test_list

In [30]:
preds_list = list()
i=1
for (tld_reseller_str in tld_reseller_list_ALL) {
    
        preds_list[[tld_reseller_str]] = cbind(
              test_list[[tld_reseller_str]],
              'pred_agg_glm_ALL'=ifelse(is.na(preds_agg_glm_ALL[[i]]), NA , preds_agg_glm_ALL[[i]]$predicted) ,
              'pred_agg_rf_ALL'= ifelse(is.na(preds_agg_rf_ALL[[i]]), NA , preds_agg_rf_ALL[[i]]$predicted),
              'pred_agg_glm'=ifelse(is.na(preds_agg_glm[[i]]), NA , preds_agg_glm[[i]]$predicted),
              'pred_agg_rf'=ifelse(is.na(preds_agg_rf[[i]]), NA , preds_agg_rf[[i]]$predicted),
              'pred_seg_glm_ALL'=ifelse(is.na(preds_seg_glm_ALL[[i]]), NA , preds_seg_glm_ALL[[i]]$predicted),
              'pred_seg_rf_ALL' = ifelse(is.na(preds_seg_rf_ALL[[i]]), NA , preds_seg_rf_ALL[[i]]$predicted),
              'pred_seg2_glm_ALL'=ifelse(is.na(preds_seg2_glm_ALL[[i]]), NA , preds_seg2_glm_ALL[[i]]$predicted),
              'pred_seg2_rf_ALL'=ifelse(is.na(preds_seg2_rf_ALL[[i]]), NA , preds_seg2_rf_ALL[[i]]$predicted))
    
        preds_list[[tld_reseller_str]] <- preds_list[[tld_reseller_str]] %>% select(-contains(".actual"))
        names(preds_list[[tld_reseller_str]]) <- c(names(test_list[[tld_reseller_str]]),
                                                   c('pred_agg_glm_ALL', 'pred_agg_rf_ALL', 
                                                   'pred_agg_glm','pred_agg_rf',
                                                   'pred_seg_glm_ALL', 'pred_seg_rf_ALL',
                                                   'pred_seg2_glm_ALL','pred_seg2_rf_ALL'))
    i=i+1
}

In [32]:
na.omit.list <- function(y) { return(y[!sapply(y, function(x) all(is.na(x)))]) }
preds_list <- na.omit.list(preds_list)
preds_df <- rbindlist(preds_list,use.names=TRUE)

In [33]:
preds_df

renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,log_reg_arpt,tld_registrar_index,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL
<chr>,<int>,<date>,<date>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
FirstTime,1,2019-06-14,2018-06-14,Active,70181780,steibei.space,Internet GmbH,Internet GmbH,Germany,NA,1,6.00,6.0000,0,208493821,auto-renewal,realized,Renewed,2019-06-14,15.0,4.02,steibei,1,1,7,l,7l,space,1.79175947,spaceinternet gmbh,0.920,0.9713886,NA,0.7251219,0.975,0.98133698,NA,NA
FirstTime,1,2020-05-17,2019-05-17,Active,103906916,wingtsun.space,Internet GmbH,Internet GmbH,Germany,NA,1,15.00,15.0000,0,315625244,auto-renewal,realized,Renewed,2020-05-17,15.0,6.48,wingtsun,1,1,8,l,8l,space,2.70805020,spaceinternet gmbh,0.924,0.9685702,NA,0.6956184,0.993,0.97838698,NA,NA
FirstTime,1,2019-10-05,2018-10-05,Active,78360794,sfmart.store,PDR Ltd,Arvixe,United States,NA,1,6.50,6.5000,0,235884471,renewal,NA,Renewed,2019-10-01,26.5,1.15,sfmart,1,1,6,l,6l,store,1.87180218,storearvixe,0.000,0.2597214,NA,0.7862930,NA,NA,NA,NA
FirstTime,1,2020-08-30,2019-08-30,Active,122049036,qianxiyun.online,,,,NA,1,25.00,25.0000,0,341703158,auto-renewal,unrealized,Not Renewd,2020-08-30,6.0,0.63,qianxiyun,1,1,9,l,9l,online,3.21887582,online,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA
FirstTime,1,2019-08-15,2018-08-15,Active,74267354,ontarioart.space,Register.ca,Register.ca,Canada,NA,1,6.00,6.0000,0,222880706,auto-renewal,unrealized,Not Renewd,2019-08-15,15.0,6.11,ontarioart,1,1,10,l,10l,space,1.79175947,spaceregister.ca,NA,0.9855667,NA,0.7286549,NA,NA,NA,NA
FirstTime,1,2020-04-22,2019-04-22,Active,101141084,ziprecruiter.press,NICENIC,NICENIC,Hong Kong,NA,1,1.08,1.0800,0,295236384,transfer,NA,Renewed,2020-02-26,49.0,5.61,ziprecruiter,1,1,12,l,12l,press,0.07696104,pressnicenic,0.022,0.2796425,NA,0.5721173,0.093,0.30130160,NA,NA
FirstTime,1,2019-06-27,2018-06-27,Active,71066905,allmaker.space,1&1 Internet,Arsys,Spain,NA,1,1.00,1.0000,0,211876061,auto-renewal,unrealized,Not Renewd,2019-06-27,6.0,6.55,allmaker,1,1,8,l,8l,space,0.00000000,spacearsys,0.487,0.5107410,NA,0.5963515,0.544,0.83162583,NA,NA
FirstTime,1,2019-11-19,2018-11-19,Active,83956614,frogjuggler.online,Imperial Registrations,Imperial Registrations,United States,NA,1,25.00,25.0000,0,256244725,auto-renewal,unrealized,Not Renewd,2019-11-19,25.0,2.98,frogjuggler,1,1,11,l,11l,online,3.21887582,onlineimperial registrations,NA,0.9948964,NA,0.8257238,NA,NA,NA,NA
FirstTime,1,2020-08-07,2019-08-07,Active,116039006,fh1.site,NetClues,NetClues,,NA,1,0.00,0.0001,0,335679829,auto-renewal,realized,Renewed,2020-08-07,20.0,0.02,fh1,1,1,3,ln,3ln,site,-9.21034037,sitenetclues,0.000,0.1291702,NA,0.5810059,NA,NA,NA,NA


In [34]:
write.csv(preds_df, file=file.path(fullDir,'preds','preds.csv'),row.names = FALSE)

# Rehash on 52 tld-re's

In [6]:
fullDir='/home/jupyter/Domains_202003/data/output/models_20201104'

In [9]:
tld_reseller_list1 = expiry_train_df %>%  distinct(tld_registrar_index) %>% pull(tld_registrar_index)
tld_reseller_list2 = expiry_test_df %>%  distinct(tld_registrar_index) %>% pull(tld_registrar_index)
tld_reseller_list = setdiff(tld_reseller_list2,tld_reseller_list1)
tld_registrar_excl_list = tld_registrar_excl(train_list = expiry_train_list)
cat("Processing", length(tld_reseller_list),"tld-re's\n")

Excluding 1706 tld-re's (80.89%), due to volume < 500 
Excluding 157 tld-re's (7.44%) associated with tld's 'pw' and 'in.net' 
Processing 52 tld-re's


In [13]:
test_list = expiry_test_list
tld_reseller_str = tld_reseller_list[[1]]
tld_reseller_str

[1] "spaceinternet gmbh"

### seg2models

In [20]:
# obviously, seg2 would all be na because models at the tld-registrara level don't exist

### segmodels

In [23]:
test_list_tld_reseller = test_list[tld_reseller_str]
test_df_tld_reseller =  rbindlist(test_list_tld_reseller,use.names=TRUE)

In [4]:
# reseller_str = test_df_tld_reseller %>% filter(tld_registrar_index==tld_reseller_str) %>% distinct(reseller) %>% pull(reseller)
reseller_str = 'Internet GmbH'

In [8]:
model_name <- paste0('model_seg_rf_',str_replace_all(reseller_str, "[^[:alnum:]]", ""))
model_name
file.path(fullDir,paste0(model_name,'.Rdata'))
load(file.path(fullDir,paste0(model_name,'.Rdata')))

[1] "model_seg_rf_InternetGmbH"

[1] "/home/jupyter/Domains_202003/data/output/models_20201104/model_seg_rf_InternetGmbH.Rdata"

In [9]:
exists(model_name)

[1] TRUE

In [37]:
model_seg_rf_InternetGmbH

Ranger result

Call:
 ranger(formula = renewal_status ~ pattern_domain_count + log_reg_arpt +      sld_length + gibb_score + sld_type + day_domains + reg_period +      tld, data = train_df_reseller, importance = "impurity", num.trees = 1000,      probability = TRUE, replace = FALSE, sample.fraction = sample_fraction,      seed = 123, respect.unordered.factors = TRUE) 

Type:                             Probability estimation 
Number of trees:                  1000 
Sample size:                      34 
Number of independent variables:  8 
Mtry:                             2 
Target node size:                 10 
Variable importance mode:         impurity 
Splitrule:                        gini 
OOB prediction error (Brier s.):  0.05751451 

In [42]:
preds_seg_rf = pred_seg_rf(test_list, tld_reseller_str)

[1] "spaceinternet gmbh"


In [43]:
preds_seg_rf

actual,predicted
<fct>,<dbl>
Renewed,0.981337
Renewed,0.978387


# DEFINE limited global objects

In [61]:
# tld_reseller_list = expiry_train_df_1 %>% distinct(tld_registrar_index) %>% 
#   pull(tld_registrar_index) 
# length(tld_reseller_list)
# tld_reseller_list = c('storenetwork solutions','storeru-center','storebigrock','onlinewhois','websiteregister.it')
#tld_reseller_list

[1] 1723

# Exclude low-vol tld-re's

In [6]:
tld_registrar_excl_list = tld_registrar_excl(train_list = expiry_train_prepped_2_1)

Exluding 1239 tld-re's (71.91%), due to volume < 100 


In [90]:
# tld_reseller_list = tld_reseller_list[!(tld_reseller_list %in% tld_registrar_exl_list)]
# length(tld_reseller_list)

[1] 5

In [7]:
tld_reseller_list = c('websitenamecheap','storeuol','storemat bao','onlinedonweb','onlinecrazy domains')

In [8]:
train_list = expiry_train_prepped_2_1[tld_reseller_list]

In [9]:
length(train_list)

[1] 5

# TEST train_all in functions*.R

In [10]:

source('functions_metalearning.R')

In [ ]:
# subDir = paste("models", format(Sys.Date(), format="%Y%m%d") , sep = "_")
# fullDir = file.path('../../data/output', subDir)
# dir.create(fullDir, showWarnings = FALSE)

In [11]:
tld_reseller_list = train_all(tld_reseller_list,
                              tld_registrar_excl_list,
                       train_list = expiry_train_prepped_2_1,
                       test_list = expiry_test_prepped_2_1,
                       model_agg_glm = NULL, 
                       model_agg_rf = NULL,
         fullDir='../../data/output/models_20201020')   

[1] "model_seg_glm_crazydomains"
[1] "model_seg_rf_crazydomains"
[1] "model_seg_glm_donweb"
[1] "model_seg_rf_donweb"
[1] "model_seg_glm_matbao"
[1] "model_seg_rf_matbao"
[1] "model_seg_glm_uol"
[1] "model_seg_rf_uol"
[1] "model_seg_glm_namecheap"
[1] "model_seg_rf_namecheap"
Growing trees.. Progress: 42%. Estimated remaining time: 43 seconds.
Growing trees.. Progress: 84%. Estimated remaining time: 11 seconds.
[1] "model_seg2_glm_websitenamecheap"
[1] "model_seg2_rf_websitenamecheap"
[1] "model_seg2_glm_storeuol"
[1] "model_seg2_rf_storeuol"
[1] "model_seg2_glm_onlinedonweb"
[1] "model_seg2_rf_onlinedonweb"
[1] "model_seg2_glm_onlinecrazydomains"
[1] "model_seg2_rf_onlinecrazydomains"
[1] "model_seg2_glm_storematbao"
[1] "model_seg2_rf_storematbao"


In [12]:
tld_reseller_list

[1] "websitenamecheap"    "storeuol"            "onlinedonweb"       
[4] "onlinecrazy domains" "storemat bao"

# TEST pred_all in functions*.R

In [13]:
source('functions_metalearning.R')

In [15]:
preds_df <- pred_all(tld_reseller_list, tld_registrar_excl_list,
                      test_list = expiry_test_prepped_2_1,
                  fullDir='../../data/output/models_20201020' # dir of models
                      )



Predicting model_agg_glm
[1] "websitenamecheap"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


[1] "storeuol"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


[1] "onlinedonweb"


...

Warning message in if (is.na(model) | is.null(test_data)) {:
“the condition has length > 1 and only the first element will be used”


[1] "storemat bao"


Warning message in if (is.na(model) | is.null(test_data)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”




Predicting model_seg2_rf
[1] "websitenamecheap"
[1] "storeuol"
[1] "onlinedonweb"
[1] "onlinecrazy domains"
[1] "storemat bao"


Warning message in if (is.na(preds_seg_glm[[i]])) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(preds_seg_glm[[i]])) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(preds_seg_glm[[i]])) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(preds_seg_glm[[i]])) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(preds_seg_glm[[i]])) {:
“the condition has length > 1 and only the first element will be used”


In [16]:
preds_df %>% select(renewal_status,contains("pred")) %>% head()

renewal_status,pred_agg_glm,pred_agg_rf,pred_seg_glm,pred_seg_rf,pred_seg2_glm,pred_seg2_rf
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Renewed,0.193,0.4870859,0.719,0.4438893,0.3787142,0.5009564
Not Renewd,0.191,0.3272803,0.716,0.3856979,0.3737353,0.4038157
Not Renewd,0.201,0.4503606,0.732,0.4557063,0.3911853,0.4264446
Not Renewd,0.188,0.4750480,0.712,0.5444202,0.3685513,0.4542695
Not Renewd,0.188,0.3215509,0.711,0.4049546,0.3692081,0.4116593
Not Renewd,0.183,0.3699182,0.701,0.4838001,0.3612048,0.4254576


In [17]:
preds_df %>% select(renewal_status,contains("pred")) %>% tail()

renewal_status,pred_agg_glm,pred_agg_rf,pred_seg_glm,pred_seg_rf,pred_seg2_glm,pred_seg2_rf
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Not Renewd,NA,NA,NA,NA,NA,NA
Not Renewd,NA,NA,NA,NA,NA,NA
Not Renewd,NA,NA,NA,NA,NA,NA
Not Renewd,NA,NA,NA,NA,NA,NA
Not Renewd,NA,NA,NA,NA,NA,NA
Not Renewd,NA,NA,NA,NA,NA,NA


# Fallback 
Since low volume tld-re's were excluded from training and testing, their predictions will be NA, so we just fill all NAs with fallback values.

In [135]:
source('functions_fallback.R')

In [35]:
# THIS IS NO LONGER NEEDED (SEE ABOVE)
# # define binary indicator for fallback use
# preds_df <- preds_df %>% mutate(pred_agg_glm_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_agg_glm), 1,0),
#                                pred_agg_rf_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_agg_rf), 1,0),
#                                pred_seg_glm_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_seg_glm), 1,0),
#                                pred_seg_rf_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_seg_rf), 1,0),
#                                pred_seg2_glm_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_seg2_glm), 1,0),
#                                pred_seg2_rf_fb = ifelse((tld_registrar_index %in% tld_registrar_fb_list) | 
#                                              is.na(pred_seg2_rf), 1,0),
#                                )
   
# # preds_df %>% group_by(tld_registrar_index, pred_agg_glm_fb) %>% tally()
# # preds_df %>% group_by(tld_registrar_index, pred_agg_rf_fb) %>% tally()
# # preds_df %>% group_by(tld_registrar_index, pred_seg_glm_fb) %>% tally()
# # preds_df %>% group_by(tld_registrar_index, pred_seg_rf_fb) %>% tally()
# # preds_df %>% group_by(tld_registrar_index, pred_seg2_glm_fb) %>% tally() 
# # preds_df %>% group_by(tld_registrar_index, pred_seg2_rf_fb) %>% tally()

In [111]:
# system("gsutil cp gs://data_input/PredictiveModelAnalysis_ResellerGeoMap.csv /home/jupyter/local/Domains_202003/data/PredictiveModelAnalysis_ResellerGeoMap.csv")
geoLookupDF <- read.csv("/home/jupyter/local/Domains_202003/data/PredictiveModelAnalysis_ResellerGeoMap.csv")
# need to remove final row with all NAs
geoLookupDF <- geoLookupDF %>% filter_all(any_vars(!is.na(.)))


In [133]:
geo_lookup <- function(geoLookupDF,
                         preds_df) {
        
    # 1. use geo lookup with duplicate rows removed
    #    remove duplicate rows in geo_lookup due to registrar level segmentation
    #    results in dims matching for expiry_test

    geoLookupDF <- geoLookupDF %>% distinct(reseller,reseller_country, reseller_geo)

    preds_df$reseller <- factor(preds_df$reseller)
    preds_df$reseller_country <- factor(preds_df$reseller_country)
    preds_df <- as.data.frame(preds_df)

    preds_df <- merge(preds_df,
                      geoLookupDF,
                      on=c('reseller','reseller_country'), 
                      all.x = TRUE)
    
    
    
    # 2. create a new lookup where we drop everything except for 
    #    reseller_country and _geo and have NA map to Others
    
    geoLookupDF <- geoLookupDF %>% distinct(reseller_country, reseller_geo) %>% 
                    mutate(reseller_geo = as.character(reseller_geo)) %>%
                    mutate(eseller_geo = if_else(is.na(reseller_country), 'Others', reseller_geo) ) %>% 
                    distinct(reseller_country, reseller_geo) %>% 
                    mutate(reseller_geo = as.factor(reseller_geo))

    # use new lookup to fill reseller_geo based just on reseller_country
    preds_df[['reseller_geo']][is.na(preds_df[['reseller_geo']])]<-
                    geoLookupDF$reseller_geo[match(
                        preds_df$reseller_country[is.na(preds_df[['reseller_geo']])],
                        geoLookupDF$reseller_country)]

    # manual fix for reseller geo 
    preds_df[['reseller_geo']][preds_df[['reseller_country']]=='Southafrica']<-'South Africa'

    # Print remaining NA reseller_geos
    rem_res <- preds_df %>% filter(is.na(reseller_geo)) %>% 
          distinct(reseller,reseller_country, reseller_geo) %>% pull(reseller)
    cat("Resellers with unmatched reseller_geo's: ",paste0(rem_res, sep=", "))
    
    return(preds_df)
}

In [134]:
preds_df_2 = geo_lookup(geoLookupDF,preds_df)


Resellers with unmatched reseller_geo's:  net4 india,  virtua drug, 

In [138]:
preds_df_2 %>%
  select(everything()) %>%  # replace to your needs
  summarise_all(funs(sum(is.na(.)))) %>% t()

Warning message:
“`funs()` is deprecated as of dplyr 0.8.0.
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


reseller,0
reseller_country,0
renewal_type,0
renewed_count,0
expiry_date,0
domain_id,0
domain,0
creation_date,0
status,0
tld,0
registrar,0


In [136]:
npv_fallback_list = fallback_gen( npv_historic_renewal_data = expiry_train_df_1, reseller_am_geo_map = geoLookupDF)

`summarise()` regrouping output by 'tld', 'reseller_geo' (override with `.groups` argument)

`summarise()` regrouping output by 'reseller_geo' (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` regrouping output by 'region' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



In [137]:
# return list members to objects
names(npv_fallback_list)
for(i in 1:length(npv_fallback_list)) assign(names(npv_fallback_list)[i], npv_fallback_list[[i]])

[1] "npv_fallback_first_geo_arpt_tld" "npv_fallback_first_geo_arpt"    
 [3] "npv_fallback_first_tld_arpt"     "npv_fallback_second_tld_geo"    
 [5] "npv_fallback_second_geo"         "npv_fallback_third_tld_geo"     
 [7] "npv_fallback_third_geo"          "npv_fallback_first_final"       
 [9] "npv_fallback_second_final"       "npv_fallback_third_final"

In [143]:
cols <- preds_df %>% select(contains("pred")) %>% names()

In [145]:
preds_df_3 <- preds_df_2

for (col in cols) {
    preds_df_3 <- fallback_app_1(
                test_data_op=preds_df_3,
                in_col=col,
                out_col=paste0(col,'_fb'))
}


dim(preds_df_2)
dim(preds_df_3)

[1] 11370    46

[1] 11370    53

In [146]:
preds_df_3 %>%
  select(everything()) %>%  # replace to your needs
  summarise_all(funs(sum(is.na(.)))) %>% t()

reseller,0
reseller_country,0
renewal_type,0
renewed_count,0
expiry_date,0
domain_id,0
domain,0
creation_date,0
status,0
tld,0
registrar,0


# Scope out load_prep functions

In [15]:
# load data
expiry_df <- readRDS("/home/jupyter/local/Domains_202003/data/output/expiry_20200701_20201015")
dim(expiry_df)
str(expiry_df)

[1] 1452307      27

tibble [1,452,307 × 27] (S3: tbl_df/tbl/data.frame)
 $ renewal_type        : chr [1:1452307] "Second" "FirstTime" "FirstTime" "Second" ...
 $ renewed_count       : int [1:1452307] 2 1 1 2 1 3 2 1 1 1 ...
 $ expiry_date         : Date[1:1452307], format: "2020-08-28" "2020-08-28" ...
 $ creation_date       : Date[1:1452307], format: "2018-08-28" "2018-08-28" ...
 $ domain_id           : int [1:1452307] 75268249 75227889 119902346 75269329 119873126 51570563 75278149 119958381 119912246 119832541 ...
 $ domain              : chr [1:1452307] "grafenhaus.online" "pultech.tech" "lebedeva-olga.space" "rvparts.site" ...
 $ registrar           : chr [1:1452307] "United Domains" "Domain.com - Registrar" "PDR Ltd" "Namecheap" ...
 $ reseller            : chr [1:1452307] "United Domains" "Domain.com - Registrar" "Timeweb" "Namecheap" ...
 $ reseller_country    : chr [1:1452307] "Germany" "United States" "Russia" "United States" ...
 $ registrant_country  : chr [1:1452307] NA NA NA NA ...
 $ reg_p

In [17]:
necessary_cols <- c('pattern_domain_count','reg_arpt','sld_length','gibb_score',
                    'sld_type','day_domains','reg_period','reseller')
expiry_df %>% select(all_of(necessary_cols)) %>% head()

pattern_domain_count,reg_arpt,sld_length,gibb_score,sld_type,day_domains,reg_period,reseller
<int>,<dbl>,<int>,<dbl>,<chr>,<int>,<int>,<chr>
1,10.0,10,3.62,l,15,1,United Domains
2,11.5,7,5.61,l,7,2,Domain.com - Registrar
1,0.2,13,2.41,hyphen-l,4,1,Timeweb
1,12.5,7,1.60,l,847,1,Namecheap
2,50.0,5,6.00,l,2,1,United Domains
1,12.5,4,5.50,l,71,1,Namecheap


In [18]:
# expiry_df <- expiry_df %>% mutate (reg_arpt = ifelse(reg_arpt <= 0, 0.0001,reg_arpt),
#                                    log_reg_arpt = log(reg_arpt),
#                                    tld_registrar_index = tolower(paste(tld, reseller,sep="")))
# str(expiry_df)
# head(expiry_df)

ERROR: Error: Problem with `mutate()` input `tld_registrar_index`.
[31m✖[39m object 'tld' not found
[34mℹ[39m Input `tld_registrar_index` is `tolower(paste(tld, reseller, sep = ""))`.


In [6]:
summary(expiry_df$log_reg_arpt)
sum(is.na(expiry_df$log_reg_arpt))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-9.2103 -0.9163 -0.5276 -0.8236  0.4055  4.1744 

[1] 0

## missing data
Training model_agg_rf_ALL
Error: Missing data in columns: pattern_domain_count, sld_length, gibb_score, sld_type, day_domains.

In [19]:
expiry_df %>%
  select(everything()) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

renewal_type,0
renewed_count,0
expiry_date,0
creation_date,3084
domain_id,0
domain,0
registrar,0
reseller,0
reseller_country,0
registrant_country,1451243
reg_period,0


In [34]:
expiry_df %>% filter(is.na(gibb_score)) %>% filter(renewed_count==1) %>% select(pattern_domain_count, sld_length, gibb_score, sld_type, day_domains) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

pattern_domain_count,593
sld_length,593
gibb_score,593
sld_type,593
day_domains,593


In [22]:
expiry_df %>% filter(is.na(gibb_score)) %>% summarise(min(expiry_date), max(expiry_date))

min(expiry_date),max(expiry_date)
<date>,<date>
2020-07-01,2020-10-15


In [33]:

library(lubridate)
expiry_df %>% filter(is.na(gibb_score)) %>% filter(renewed_count==1) %>% group_by(month=floor_date(expiry_date, "month")) %>%
   summarize(count=n())

`summarise()` ungrouping output (override with `.groups` argument)



month,count
<date>,<int>
2020-07-01,107
2020-08-01,174
2020-09-01,222
2020-10-01,90


In [25]:
expiry_train_df_1  %>%  filter(is.na(gibb_score)) %>% select(pattern_domain_count, sld_length, gibb_score, sld_type, day_domains) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

pattern_domain_count,0
sld_length,0
gibb_score,0
sld_type,0
day_domains,0


In [27]:
names(expiry_df)

[1] "renewal_type"         "renewed_count"        "expiry_date"         
 [4] "creation_date"        "domain_id"            "domain"              
 [7] "registrar"            "reseller"             "reseller_country"    
[10] "registrant_country"   "reg_period"           "reg_revenue"         
[13] "reg_arpt"             "renew_mbg"            "renewal_item_id"     
[16] "renew_type"           "autorenew_type"       "renewal_status"      
[19] "renew_date"           "renew_arpt"           "gibb_score"          
[22] "pattern"              "pattern_domain_count" "day_domains"         
[25] "sld_length"           "sld_type"             "sld_type2"

In [28]:
expiry_df %>% filter(is.na(gibb_score)) %>% filter(floor_date(expiry_date, "month")=='2020-10-01') %>% group_by(renewed_count) %>% tally()

renewed_count,n
<int>,<int>
1,90
2,275
3,497
4,3968
5,2941
6,649
7,252


In [32]:
expiry_df %>% filter(is.na(gibb_score)) %>% filter(floor_date(expiry_date, "month")=='2020-10-01') %>% 
    group_by(floor_date(creation_date, "month") ) %>% tally()

"floor_date(creation_date, ""month"")",n
<date>,<int>
2013-10-01,271
2014-10-01,756
2015-10-01,3364
2016-10-01,4253
NA,28


# Check -- did we exclude renewed_count>1 from previous analyses?

In [6]:
names(expiry_train_df_1)

[1] "renewal_type"         "renewed_count"        "expiry_date"         
 [4] "domain_id"            "domain"               "creation_date"       
 [7] "status"               "tld"                  "registrar"           
[10] "reseller"             "reseller_country"     "region"              
[13] "reg_period"           "registrant_country"   "renewal_status"      
[16] "renew_mbg"            "renew_type"           "autorenew_type"      
[19] "renew_date"           "renew_registrar"      "renew_reseller"      
[22] "reg_revenue"          "reg_arpt"             "renew_period"        
[25] "renew_domain_revenue" "renew_arpt"           "reg_arpt_org"        
[28] "tld_registrar_index"  "sld"                  "sld_type"            
[31] "sld_length"           "sld_type2"            "day_domains"         
[34] "log_reg_arpt"         "gibb_score"           "pattern"             
[37] "cluster"              "pattern_score"        "pattern_domain_count"

In [7]:
expiry_train_df_1 %>% group_by(renewed_count) %>% tally()

renewed_count,n
<int>,<int>
1,1188250


# Last check in before script run 20201028

In [4]:
library(dplyr)

In [27]:
expiry_df <- readRDS("/home/jupyter/local/Domains_202003/data/output/expiry_20190601_20200901")

In [28]:
dim(expiry_df)
expiry_df %>% summarise(max(expiry_date), min(expiry_date))

[1] 5455386      28

max(expiry_date),min(expiry_date)
<date>,<date>
2020-09-01,2019-06-01


In [29]:
expiry_df <- expiry_df %>% filter(expiry_date >= as.Date("2019-06-01") & expiry_date <= as.Date("2020-09-01"))
dim(expiry_df)
expiry_df %>% summarise(max(expiry_date), min(expiry_date))

[1] 5455386      28

max(expiry_date),min(expiry_date)
<date>,<date>
2020-09-01,2019-06-01


In [30]:
expiry_df %>% group_by(renewed_count) %>% tally()

renewed_count,n
<int>,<int>
1,4688060
2,366856
3,200646
4,137897
5,50449
6,7628
7,3662
8,188


In [31]:
expiry_df <- expiry_df %>% filter(renewed_count==1)
dim(expiry_df)
expiry_df %>% summarise(max(expiry_date), min(expiry_date))
expiry_df %>% group_by(renewed_count) %>% tally()

[1] 4688060      28

max(expiry_date),min(expiry_date)
<date>,<date>
2020-09-01,2019-06-01


renewed_count,n
<int>,<int>
1,4688060


In [32]:
expiry_df %>%
  select(everything()) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

renewal_type,0
renewed_count,0
expiry_date,0
creation_date,0
domain_id,0
domain,0
registrar,0
reseller,0
reseller_country,0
registrant_country,4680297
reg_period,0


In [33]:
library(lubridate)
expiry_df %>% filter(is.na(gibb_score)) %>% 
    summarise(max(creation_date))

max(creation_date)
<date>
2016-12-31


In [34]:
names(expiry_df)

[1] "renewal_type"         "renewed_count"        "expiry_date"         
 [4] "creation_date"        "domain_id"            "domain"              
 [7] "registrar"            "reseller"             "reseller_country"    
[10] "registrant_country"   "reg_period"           "reg_revenue"         
[13] "reg_arpt"             "renew_mbg"            "renewal_item_id"     
[16] "renew_type"           "autorenew_type"       "renewal_status"      
[19] "renew_date"           "renew_arpt"           "gibb_score"          
[22] "pattern"              "pattern_domain_count" "day_domains"         
[25] "sld_length"           "sld_type"             "sld_type2"           
[28] "tld"

In [35]:
expiry_df %>% filter(is.na(gibb_score)) %>% tally() %>% pull(n)

[1] 7748

In [36]:
library(lubridate)
cat("Removed", expiry_df %>% filter(is.na(gibb_score)) %>% tally() %>% pull(n) ,"rows due to missing gibb_score")
expiry_df <- expiry_df %>% filter(!is.na(gibb_score))
cat("... now dataset min(creation_date) is ", expiry_df %>% summarise(min(creation_date)) %>% pull(1) %>% as.character(),".")
dim(expiry_df)

Removed 7748 rows due to missing gibb_score... now dataset min(creation_date) is  2017-01-01 .

[1] 4680312      28

In [37]:
4688060-7748

[1] 4680312

In [38]:
expiry_df %>%
  select(everything()) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

renewal_type,0
renewed_count,0
expiry_date,0
creation_date,0
domain_id,0
domain,0
registrar,0
reseller,0
reseller_country,0
registrant_country,4680294
reg_period,0


In [39]:
expiry_df <- expiry_df %>% mutate (reg_arpt = ifelse(reg_arpt <= 0, 0.0001,reg_arpt),
                                   log_reg_arpt = log(reg_arpt),
                                   tld_registrar_index = tolower(paste(tld, reseller,sep="")))

In [40]:
expiry_df %>%
  select(everything()) %>% 
  summarise_all(funs(sum(is.na(.)))) %>% t()

renewal_type,0
renewed_count,0
expiry_date,0
creation_date,0
domain_id,0
domain,0
registrar,0
reseller,0
reseller_country,0
registrant_country,4680294
reg_period,0


In [41]:
expiry_df %>% filter(is.na(reg_arpt)) %>% pull(log_reg_arpt)

numeric(0)

In [45]:
cat(paste0(c("hi","bye"), sep=","))

hi, bye,

# QA script fails

## check: predictions for model_agg_glm_ALL != model_agg_glm for a given tld_registrar

In [175]:
fullDir <- '/home/jupyter/local/Domains_202003/data/output/models_20201028'
preds <- read.csv(file.path(fullDir,'preds.csv'))

In [176]:
head(preds)

,renewal_type,renewed_count,expiry_date,creation_date,domain_id,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,log_reg_arpt,tld_registrar_index,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL
,<fct>,<int>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FirstTime,1,2020-06-30,2019-06-30,110777246,entai.space,OVH,OVH,France,NA,1,0.5,0.5,0,326051494,auto-renewal,unrealized,Not Renewd,2020-06-30,15,6.29,entai,1,3,5,l,5l,space,-0.6931472,spaceovh,0.382,0.27858307,0.382,0.27858307,0.386,0.28259150,0.4064797,0.29621531
2,FirstTime,1,2020-05-13,2019-05-13,103425809,bxpgdkx.space,OVH,OVH,France,NA,1,0.5,0.5,0,314864189,auto-renewal,unrealized,Not Renewd,2020-05-13,15,0.11,bxpgdkx,1,33,7,l,7l,space,-0.6931472,spaceovh,0.238,0.04722732,0.238,0.04722732,0.266,0.03662617,0.1304821,0.04210570
3,FirstTime,1,2020-05-13,2019-05-13,103426079,pirzkla.space,OVH,OVH,France,NA,1,0.5,0.5,0,314864309,auto-renewal,unrealized,Not Renewd,2020-05-13,15,1.32,pirzkla,1,33,7,l,7l,space,-0.6931472,spaceovh,0.241,0.04389334,0.241,0.04389334,0.270,0.05025404,0.1334309,0.04561117
4,FirstTime,1,2020-05-13,2019-05-13,103450484,maboutique.space,OVH,OVH,France,NA,1,0.5,0.5,0,314864449,auto-renewal,unrealized,Not Renewd,2020-05-13,15,5.81,maboutique,1,33,10,l,10l,space,-0.6931472,spaceovh,0.251,0.22431104,0.251,0.22431104,0.288,0.23679877,0.1479498,0.22100320
5,FirstTime,1,2020-05-13,2019-05-13,103406354,bassguitar.space,OVH,OVH,France,NA,1,0.5,0.5,0,314864074,auto-renewal,unrealized,Not Renewd,2020-05-13,15,3.43,bassguitar,1,33,10,l,10l,space,-0.6931472,spaceovh,0.245,0.21018523,0.245,0.21018523,0.281,0.20246193,0.1416794,0.20269325
6,FirstTime,1,2020-07-03,2019-07-03,111402996,flunkerei.space,OVH,OVH,France,NA,1,0.5,0.5,0,312912559,renewal,NA,Renewed,2020-05-04,15,5.20,flunkerei,1,8,9,l,9l,space,-0.6931472,spaceovh,0.354,0.42977486,0.354,0.42977486,0.371,0.43582715,0.3537241,0.41850528


In [177]:
names(preds)

[1] "renewal_type"         "renewed_count"        "expiry_date"         
 [4] "creation_date"        "domain_id"            "domain"              
 [7] "registrar"            "reseller"             "reseller_country"    
[10] "registrant_country"   "reg_period"           "reg_revenue"         
[13] "reg_arpt"             "renew_mbg"            "renewal_item_id"     
[16] "renew_type"           "autorenew_type"       "renewal_status"      
[19] "renew_date"           "renew_arpt"           "gibb_score"          
[22] "pattern"              "pattern_domain_count" "day_domains"         
[25] "sld_length"           "sld_type"             "sld_type2"           
[28] "tld"                  "log_reg_arpt"         "tld_registrar_index" 
[31] "pred_agg_glm_ALL"     "pred_agg_rf_ALL"      "pred_agg_glm"        
[34] "pred_agg_rf"          "pred_seg_glm_ALL"     "pred_seg_rf_ALL"     
[37] "pred_seg2_glm_ALL"    "pred_seg2_rf_ALL"

In [180]:
summary(as.data.frame(preds$pred_agg_rf == preds$pred_agg_rf_ALL ))

 preds$pred_agg_rf == preds$pred_agg_rf_ALL
 Mode:logical                              
 TRUE:3498                                 
 NA's:29695                                

In [181]:
summary(as.data.frame(preds$pred_agg_glm == preds$pred_agg_glm_ALL ))

 preds$pred_agg_glm == preds$pred_agg_glm_ALL
 Mode:logical                                
 TRUE:3498                                   
 NA's:29695                                  

In [165]:
# pred = mass_predict_first_renewal(test_list_tld_reseller, model)
test_data = test_list_tld_reseller

  tld_registrar_list<-names(test_data)
  tld_registrar_list <- tld_registrar_list[!is.na(tld_registrar_list)]
  prediction_list<-pblapply(tld_registrar_list, function(i) list_predict_first_renewal(i, test_data, data_models))
  prediction_list<-prediction_list[!is.na(prediction_list)]
  prediction_op<-rbindlist(prediction_list,use.names=TRUE, fill=TRUE) 
  test_data<-rbindlist(test_data,  use.names=TRUE, fill=TRUE) # modified as per error on 20200715 (see README)
  test_data$first_renewal_prediction<-prediction_op$probabilities[match(test_data$domain_id,
                                                                        prediction_op$domain_id)]

Warning message in if (is.na(model) | is.null(test_data)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”
Warning message:
“Unknown or uninitialised column: `first_renewal_prediction`.”
Warning message:
“Unknown or uninitialised column: `Status`.”


ERROR: Error: Assigned data `*vtmp*` must be compatible with existing data.
[31m✖[39m Existing data has 503 rows.
[31m✖[39m Assigned data has 0 rows.
[34mℹ[39m Only vectors of size 1 are recycled.


# CHeck output

In [4]:
preds <- read.csv('../../data/output/models_20201102/preds.csv')

In [5]:
dim(preds)

[1] 932289     39

In [6]:
head(preds)

,renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,log_reg_arpt,tld_registrar_index,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL
,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FirstTime,1,2019-11-08,2018-11-08,Active,82167309,viralherbs.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,248714055,auto-renewal,realized,Renewed,2019-11-08,7.5,3.39,viralhealh,2,576,10,l,10l,pw,-0.7339692,pwnamecheap,0.030,0.027366883,0.030,0.027366883,0.049,0.02752752,0.02812901,0.024447474
2,FirstTime,1,2019-08-05,2018-08-05,Active,73624034,ww3n.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,219523016,auto-renewal,unrealized,Not Renewd,2019-08-05,0.0,0.52,ww3n,1,257,4,ln,4ln,pw,-0.7339692,pwnamecheap,0.024,0.026322543,0.024,0.026322543,0.062,0.02930435,0.07347816,0.025803420
3,FirstTime,1,2019-08-11,2018-08-11,Active,74035654,dpasseresp.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,221379481,auto-renewal,unrealized,Not Renewd,2019-08-11,7.0,4.65,pssders,3,353,10,l,10l,pw,-0.7339692,pwnamecheap,0.035,0.006085356,0.035,0.006085356,0.066,0.01232116,0.03516253,0.006186874
4,FirstTime,1,2019-07-22,2018-07-22,Active,72660399,sexgirls.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,216639126,auto-renewal,unrealized,Not Renewd,2019-07-22,7.0,1.75,xgirl,2,80,8,l,8l,pw,-0.7339692,pwnamecheap,0.048,0.080058013,0.048,0.080058013,0.098,0.06522605,0.06273875,0.059190735
5,FirstTime,1,2019-07-07,2018-07-07,Active,71566430,yucief.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,213669206,auto-renewal,unrealized,Not Renewd,2019-07-07,7.0,1.52,cief,3,1147,6,l,6l,pw,-0.7339692,pwnamecheap,0.017,0.016695967,0.017,0.016695967,0.023,0.01608117,0.02273527,0.016375413
6,FirstTime,1,2019-11-11,2018-11-11,Active,82528489,bio-x-keto.pw,Namecheap,Namecheap,United States,NA,1,0.48,0.48,0,250798210,auto-renewal,unrealized,Not Renewd,2019-11-11,7.5,2.16,bio-x-keto,1,687,10,hyphen-l,10hyphen-l,pw,-0.7339692,pwnamecheap,0.031,0.009853222,0.031,0.009853222,0.038,0.01404684,0.02211356,0.009443069


# QA script fails

In [54]:

source('../orig/functions.R')

source('../orig/functions_models.R')

In [55]:
# model_seg2_glm_ALL
tld_registrar_str = "siteregistrator domenov"
# tld_registrar_str = "onlinenominalia"

In [56]:
model = load("../../data/output/models_20201104/model_seg2_glm_siteregistratordomenov.Rdata")

In [57]:
model <- model_seg2_glm_siteregistratordomenov

In [58]:
tld_reseller_str = tld_registrar_str
test_list = expiry_test_list
test_list_tld_reseller = test_list[tld_reseller_str]
test_df_tld_reseller =  rbindlist(test_list_tld_reseller,use.names=TRUE)

In [61]:
pred = mass_predict_first_renewal(test_list_tld_reseller, model)

Warning message in if (is.na(model) | is.null(test_data)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


In [62]:
head(pred)

renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,log_reg_arpt,tld_registrar_index,first_renewal_prediction
<chr>,<int>,<date>,<date>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
FirstTime,1,2020-04-10,2019-04-10,Active,99552094,zarya.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,306634739,auto-renewal,realized,Renewed,2020-04-10,20,5.02,zarya,1,1,5,l,5l,site,0,siteregistrator domenov,1.000000e+00
FirstTime,1,2020-03-23,2019-03-23,Active,97663239,lti-l.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,284200426,renewal,NA,Renewed,2020-01-24,20,4.49,lti-l,1,1,5,hyphen-l,5hyphen-l,site,0,siteregistrator domenov,1.000000e+00
FirstTime,1,2020-06-17,2019-06-17,Active,108122146,technogroup.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,322846904,auto-renewal,unrealized,Not Renewd,2020-06-17,20,3.68,technogroup,1,1,11,l,11l,site,0,siteregistrator domenov,2.220446e-16
FirstTime,1,2020-07-18,2019-07-18,Active,113052246,get-landing.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,330408694,auto-renewal,unrealized,Not Renewd,2020-07-18,20,9.09,gettilda,2,2,11,hyphen-l,11hyphen-l,site,0,siteregistrator domenov,2.220446e-16
FirstTime,1,2020-02-21,2019-02-21,Active,94390464,xn----7sbhb9acibxt3ndg.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,270099991,renewal,NA,Renewed,2019-12-24,20,0.64,xn----7sbhb9acibxt3ndg,1,1,22,idn,22idn,site,0,siteregistrator domenov,1.000000e+00
FirstTime,1,2020-04-09,2019-04-09,Active,99411879,web-leader.site,Registrator Domenov,Registrator Domenov,Russia,NA,1,1,1,0,306369449,auto-renewal,unrealized,Not Renewd,2020-04-09,20,6.21,web-leader,1,1,10,hyphen-l,10hyphen-l,site,0,siteregistrator domenov,2.220446e-16


In [53]:
str = "onlinenominalia"
model[[str]]


Call:  glm(formula = renewal_status ~ ., family = binomial(link = "logit"), 
    data = build_data, model = FALSE, y = FALSE)

Coefficients:
         (Intercept)  pattern_domain_count          log_reg_arpt  
            -0.51143               0.12091               0.06141  
          sld_length     sld_typehyphen-ln             sld_typel  
            -0.01685               0.20799               0.03283  
          sld_typeln             sld_typen           day_domains  
            -0.36180              -7.36347              -0.02013  
          gibb_score            reg_period  
            -0.04673                    NA  

Degrees of Freedom: 1175 Total (i.e. Null);  1166 Residual
Null Deviance:	    1255 
Residual Deviance: 1203 	AIC: 1223

In [20]:
# pred = mass_predict_first_renewal(test_list_tld_reseller, model)

test_data = test_list_tld_reseller
data_models = model

# mass_predict_first_renewal<-function(test_data, data_models) {

  tld_registrar_list<-names(test_data)
  tld_registrar_list <- tld_registrar_list[!is.na(tld_registrar_list)]
#   prediction_list<-pblapply(tld_registrar_list, function(i) list_predict_first_renewal(i, test_data, data_models))
#   prediction_list<-prediction_list[!is.na(prediction_list)]
#   prediction_op<-rbindlist(prediction_list,use.names=TRUE, fill=TRUE) 
#   test_data<-rbindlist(test_data,  use.names=TRUE, fill=TRUE) # modified as per error on 20200715 (see README)
#   test_data$first_renewal_prediction<-prediction_op$probabilities[match(test_data$domain_id,
#                                                                         prediction_op$domain_id)]
#   return(test_data)
# }

In [21]:
tld_registrar_list

[1] "siteregistrator domenov"

In [22]:
tld_registrar_list

[1] "siteregistrator domenov"

In [24]:
tld_registrar = tld_registrar_list[[1]]

In [25]:
# list_predict_first_renewal<-function(tld_registrar,test_data, data_models) {
#   cat(tld_registrar)
  ########BEGIN:for substitute TLD#######################
  #registrar.name<-test.data[[tld.registrar]]$`Client Shortname`[1]
  #new.tld.registrar<-paste("site_",registrar.name,sep="")
  #new.tld.registrar<-gsub("\\s+","_",new.tld.registrar)
  #model<-data.models[[new.tld.registrar]]
#   print(tld_registrar)
  ########END:for substitute TLD#######################
  tld_registrar_data<-test_data[[tld_registrar]]
  model<-data_models[[tld_registrar]]
 

In [33]:
names(data_models)
tld_registrar

[1] "siteregistrator domenov "

[1] "siteregistrator domenov"

In [37]:
data_models[[1]]


Call:  glm(formula = renewal_status ~ ., family = binomial(link = "logit"), 
    data = build_data, model = FALSE, y = FALSE)

Coefficients:
         (Intercept)  pattern_domain_count          log_reg_arpt  
              1161.3                    NA                 492.0  
          sld_length           sld_typeidn             sld_typel  
              -258.4                6220.1                -543.4  
          sld_typeln           day_domains            gibb_score  
              2746.5                -413.6                 266.5  
          reg_period  
                  NA  

Degrees of Freedom: 15 Total (i.e. Null);  8 Residual
Null Deviance:	    12.06 
Residual Deviance: 4.55e-08 	AIC: 16

In [ ]:
   
    # LVG removed the following and replaced with NA-fill in predict_first_renewal()
#   if(is.null(tld_registrar_data) | is.null(model) | is.logical(tld_registrar_data) | is.logical(model)) { return(NA)}
    
    
  # tld.registrar.data$SLD.Type[tld.registrar.data$SLD.Type == "IDN"]<-NA
  
  tld_registrar_data<-predict_first_renewal(tld_registrar_data,model)
  return(tld_registrar_data)
}